In [ ]:
%pip install -U datasets==2.15.0
%pip install -U --force-reinstall \
             langchain==0.0.324 \
             typing_extensions==4.7.1 \
             pypdf==3.16.4


## Deploy pretrained model

In [ ]:
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()

In [ ]:
model_id, model_version = "meta-textgeneration-llama-2-7b", "2.*"

In [ ]:
from sagemaker.jumpstart.model import JumpStartModel

pretrained_model = JumpStartModel(model_id=model_id, model_version=model_version)
pretrained_predictor = pretrained_model.deploy()

## Continued pretraining

In [ ]:
s3_prefix = f"s3://jumpstart-cache-prod-{boto3.Session().region_name}/training-datasets/sec_amazon"
s3_location = f"s3://jumpstart-cache-prod-{boto3.Session().region_name}/training-datasets/sec_amazon/AMZN_2021_2022_train_js.txt"

In [ ]:
!aws s3 cp $s3_location ./

In [ ]:
import boto3
from sagemaker.jumpstart.estimator import JumpStartEstimator

model_id, model_version = "meta-textgeneration-llama-2-7b", "2.*"

estimator = JumpStartEstimator(model_id=model_id, 
                               model_version=model_version, 
                               environment={"accept_eula": "true"},
                               instance_type = "ml.g5.24xlarge")

estimator.set_hyperparameters(instruction_tuned="False", epoch="5")

estimator.fit({"training": s3_prefix})